In [2]:
from tweepy import *
import pandas as pd
import sys

In [3]:
import tweepy
import requests
from bs4 import BeautifulSoup

In [4]:
consumer_key = argv[0]
consumer_secret = argv[1]
access_key = argv[2]
access_secret = arv[3]

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

In [6]:
api = tweepy.API(auth,wait_on_rate_limit=True)

In [7]:
wikiurl = "https://en.wikipedia.org/wiki/List_of_most-followed_Twitter_accounts"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [8]:
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {'class':"wikitable"})

In [9]:
df = pd.DataFrame(pd.read_html(str(table))[0])
df.head()

,Rank,Change (monthly),Account name,Owner,Followers (millions),Occupation,Country
0,1,NaN,@BarackObama,Barack Obama,130.3,44th President of the United States,United States
1,2,NaN,@justinbieber,Justin Bieber,114.3,Musician,Canada
2,3,NaN,@katyperry[a],Katy Perry,109.5,Musician,United States
3,4,NaN,@rihanna,Rihanna,102.7,Musician and businesswoman,Barbados
4,5,NaN,@Cristiano,Cristiano Ronaldo,92.0,Footballer,Portugal


In [10]:
df.shape
df.columns

Index(['Rank', 'Change (monthly)', 'Account name', 'Owner',
       'Followers (millions)', 'Occupation', 'Country'],
      dtype='object')

In [11]:
df['Account name'][2] = '@katyperry'
df['Account name'][15] = '@twitter'
df['Account name'][17] = '@ddlovato'
df['Account name']

<ipython-input-11-317f78244bdc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Account name'][2] = '@katyperry'
<ipython-input-11-317f78244bdc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Account name'][15] = '@twitter'
<ipython-input-11-317f78244bdc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Account name'][17] = '@ddlovato'


0         @BarackObama
1        @justinbieber
2           @katyperry
3             @rihanna
4           @Cristiano
5       @taylorswift13
6            @ladygaga
7        @ArianaGrande
8        @TheEllenShow
9             @YouTube
10      @KimKardashian
11       @narendramodi
12        @selenagomez
13        @jtimberlake
14             @cnnbrk
15            @twitter
16      @britneyspears
17           @ddlovato
18          @BillGates
19                @CNN
20           @neymarjr
21            @shakira
22           @elonmusk
23        @jimmyfallon
24            @nytimes
25          @KingJames
26        @BBCBreaking
27         @MileyCyrus
28         @SrBachchan
29                @JLo
30               @NASA
31              @Oprah
32          @BrunoMars
33    @BeingSalmanKhan
34             @iamsrk
35           @PMOIndia
36      @NiallOfficial
37           @imVkohli
38        @akshaykumar
39              @Drake
40        @KylieJenner
41       @SportsCenter
42     @KevinHart4real
43         

In [28]:
df.drop([9, 14, 15, 19, 24, 26, 30, 41, 43, 46, 47], axis=0, inplace=True)

In [29]:
celebrities = {}
for account in df['Account name']: 
    statuses = api.user_timeline(account[1:], count=100)
    celebrities[account] = statuses

In [67]:
celebrities['@BarackObama']

[Status(_api=<tweepy.api.API object at 0x7fd6c22b1be0>, _json={'created_at': 'Fri Apr 30 13:00:21 +0000 2021', 'id': 1388116011367424000, 'id_str': '1388116011367424000', 'text': 'It was because of them I was able to travel to New York City later that week and tell some of the people who had a… https://t.co/vCLIaBSoaV', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/vCLIaBSoaV', 'expanded_url': 'https://twitter.com/i/web/status/1388116011367424000', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [116, 139]}]}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': 1388115964202557441, 'in_reply_to_status_id_str': '1388115964202557441', 'in_reply_to_user_id': 813286, 'in_reply_to_user_id_str': '813286', 'in_reply_to_screen_name': 'BarackObama', 'user': {'id': 813286, 'id_str': '813286', 'name': 'Barack Obama', 'screen_name': 'BarackObama', 'location': 'Was

In [30]:
for account in df['Account name']: 
    d = {"text": [], 
         "date": [], 
         "account": [],
         "followers": [], 
         "retweets":[], 
         "favorites":[]
        }
    for status in celebrities[account]: 
        d["text"].append(status.text)
        d["date"].append(status.created_at)
        d["account"].append(status.user.screen_name)
        d["followers"].append(status.user.followers_count)
        d["retweets"].append(status.retweet_count)
        d["favorites"].append(status.favorite_count)
        
    pd.DataFrame(d).to_csv("data/celebrities/twitter_text"+account+".csv")